#### How to use the Airweave API

This notebook will go through an example of how to use the Airweave API to transform data from your app into a searchable knowledge base.
Specifically, we will make an entire GitHub repository searchable in a few steps.

##### Install the Airweave SDK

In [ ]:
pip install airweave-sdk

##### Set-up the Airweave Client
To create an API key go to https://app.airweave.ai, create an account, select "API Keys" in the sidebar and click "Create Key". 


In [ ]:
from airweave import AirweaveSDK

client = AirweaveSDK(api_key="YOUR_API_KEY")

##### Let's create a collection
A collection is a searchable knowledge base composed of different data sources.

Note that the newly created collection is initially empty and does not contain any data until we explicitly add source connections to it.

More information about the collection endpoint can be found in the [API Reference](https://docs.airweave.ai/api-reference/collections/create-collection-collections-post).

In [ ]:
collection = client.collections.create(
    name="My First Collection"
)

print(collection.readable_id)

##### Let's add a source connection to our collection
A source connection is an authenticated and configured link to a data source that syncs data into the collection.

More information about the source connection endpoint can be found in the [API Reference](https://docs.airweave.ai/api-reference/source-connections/create-source-connection-source-connections-post).

See https://docs.airweave.ai/docs/connectors/github for more information on how to get a personal access token for Github.


In [ ]:
source_connection = client.source_connections.create_source_connection(
    name="My Github Connection",
    short_name="github",
    collection=collection.readable_id,
    auth_fields={
        "personal_access_token": "YOUR_PERSONAL_ACCESS_TOKEN",
        "repo_name": "YOUR_REPO_NAME" # The repo you want to sync
    },
    config_fields={
        "branch": "main" # The branch you want to sync
    },
    cron_schedule="0 8 * * 1", # sync every Monday at 8:00 AM
    sync_immediately=True
)

##### Let's wait for the data sync to complete

The code below repeatedly checks the status of the data synchronization until it changes.

In [ ]:
import time
from datetime import datetime

print("Starting data sync", end="", flush=True)
start_time = datetime.now()

while True:
    source_connection = client.source_connections.get_source_connection(
        source_connection_id=source_connection.id
    )
    
    status = source_connection.latest_sync_job_status
    
    # Show progress
    print(".", end="", flush=True)
    
    if status == "completed":
        elapsed = datetime.now() - start_time
        print(f"\nData sync completed successfully! (took {elapsed.seconds} seconds)")
        break
    elif status == "failed":
        print(f"\nData sync {status}!")
        if source_connection.latest_sync_job_error:
            print(f"Error: {source_connection.latest_sync_job_error}")
        break
    
    time.sleep(2)

##### Let's search through our collection

Ask anything about the GitHub repository you synced.

More information about the search endpoint can be found in the [API Reference](https://docs.airweave.ai/api-reference/collections/search-collection-collections-readable-id-search-get).

In [ ]:
response = client.collections.search_collection(
    readable_id=collection.readable_id,
    query="YOUR_QUERY",
    response_type="completion"
)

print(response.completion)